In [ ]:
#| hide
from nbdev.showdoc import show_doc

%load_ext autoreload
%autoreload 2

In [ ]:
#| output: false
import numpy as np
import tempfile
from pandas import DataFrame
from pathlib import Path
import pandas as pd

from spannerlib.utils import checkLogs
from spannerlib.span import Span
from spannerlib.session import Session,test_session

In [ ]:
test_session(
    """
    new uncle(str, str)
    uncle("bob", "greg")
    ?uncle(X,Y)
    """,
    pd.DataFrame(data={"X": ["bob"], "Y": ["greg"]})
)


'?uncle(X,Y)'

X,Y
bob,greg


<spannerlib.session.Session>

In [ ]:
_= test_session(
    [
    """
    new lecturer(str, str)
    lecturer("walter", "chemistry")
    lecturer("linus", "operation systems")
    lecturer("rick", "physics")

    new enrolled(str, str)
    enrolled("abigail", "chemistry")
    enrolled("abigail", "operation systems")
    enrolled("jordan", "chemistry")
    enrolled("gale", "operation systems")
    enrolled("howard", "chemistry")
    enrolled("howard", "physics")

    enrolled_in_chemistry(X) <- enrolled(X, "chemistry").
    ?enrolled_in_chemistry("jordan")
    """,
    """
    ?enrolled_in_chemistry("gale")
    """,
    """
    ?enrolled_in_chemistry(X)
    """,
    """
    enrolled_in_physics_and_chemistry(X) <- enrolled(X, "chemistry"), enrolled(X, "physics").
    ?enrolled_in_physics_and_chemistry(X)
    """,
    """
    lecturer_of(X, Z) <- lecturer(X, Y), enrolled(Z, Y).
    ?lecturer_of(X, "abigail")
    """,
    """
    gpa_doc = "abigail 100 jordan 80 gale 79 howard 60"
    student_gpas(Student, Grade) <- rgx("(\w+).*?(\d+)",$gpa_doc)->(StudentSpan, GradeSpan),
        as_str(StudentSpan)->(Student), as_str(GradeSpan)->(Grade).
    gpa_of_chemistry_students(X, Grade) <- student_gpas(X, Grade), enrolled(X, "chemistry").
    ?gpa_of_chemistry_students(X, "100")
    """],
    [
        True,
        False,
        pd.DataFrame(data={"X": ["howard", "jordan", "abigail"]}),
        pd.DataFrame(data={"X": ["howard"]}),
        pd.DataFrame(data={"X": ["linus", "walter"]}),
        pd.DataFrame(data={"X": ["abigail"]}),
    ]
)


'?enrolled_in_chemistry("jordan")'

True

'?enrolled_in_chemistry("gale")'

False

'?enrolled_in_chemistry(X)'

...

X
howard


'?lecturer_of(X,"abigail")'

X
linus
walter


'?gpa_of_chemistry_students(X,"100")'

X
abigail


In [ ]:
_=test_session(
    ["""
    jsonpath_simple_1 = "foo[*].baz"
    json_ds_simple_1  = "{'foo': [{'baz': 1}, {'baz': 2}]}"
    simple_1(X) <- json_path($json_ds_simple_1, $jsonpath_simple_1) -> (Path,X).
    ?simple_1(X)
    ""","""
    jsonpath_simple_2 = "a.*.b.`parent`.c"
    json_ds_simple_2 = "{'a': {'x': {'b': 1, 'c': 'number one'}, 'y': {'b': 2, 'c': 'number two'}}}"
    simple_2(X) <- json_path($json_ds_simple_2, $jsonpath_simple_2) -> (Path,X).
    ?simple_2(X)
    ""","""
    json_ds_advanced  = "{'foo': [{'baz': 1}, {'baz': {'foo': [{'baz': 1}, {'baz': 2}]}}]}"
    advanced(X) <- json_path($json_ds_advanced, $jsonpath_simple_1) -> (Path,X).
    ?advanced(X)
    """],
    [
        pd.DataFrame(data={"X": ['2', '1']}),
        pd.DataFrame(data={"X": ["number two", "number one"]}),
        pd.DataFrame(data={"X": ['1','''{'foo': [{'baz': 1}, {'baz': 2}]}''']})
    ]
)

'?simple_1(X)'

X
1
2


'?simple_2(X)'

X
number one
number two


'?advanced(X)'

X
1
"{'foo': [{'baz': 1}, {'baz': 2}]}"
